# Pure Python Walkthrough


Setup the environment

In [1]:
import dotenv
import os
import logfire

import tool_functions as tf

import nest_asyncio

nest_asyncio.apply()

dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")
logfire_token = os.getenv("LOGFIRE_TOKEN")


# configure logfire
logfire.configure(token=logfire_token)

# 2. Instrumenteer LangChain (automatisch loggen van chains, LLM calls, tools, etc.)
logfire.instrument()

<function logfire._internal.instrument.instrument.<locals>.decorator(func: 'Callable[P, R]') -> 'Callable[P, R]'>

Logfire project URL: ]8;id=813962;https://logfire-eu.pydantic.dev/elbrink/claude-test\https://logfire-eu.pydantic.dev/elbrink/claude-test]8;;\


1 Definieer een basis agent

In [2]:
dotenv.load_dotenv()

api_key = os.getenv("ANTHROPIC_API_KEY")
model = os.getenv("DEFAULT_MODEL")

import json
from anthropic import Anthropic
from typing import Dict, List, Optional, Callable, Any


class LLMAgent:
    def __init__(self, name: str, system_prompt: str = ""):
        """
        Initialiseer de LLM-agent.

        Args:
            name: Naam van de agent
            system_prompt: Systeeminstructies voor de agent
        """
        self.client = Anthropic(api_key=api_key)
        self.model = model
        self.name = name
        self.system_prompt = system_prompt

    def execute_tool(self, tool_name: str, **kwargs) -> str:
        """
        Voer een tool uit met de gegeven parameters.
        """
        if tool_name in self.tools:
            try:
                result = self.tools[tool_name]["function"](**kwargs)
                return f"Tool '{tool_name}' executed successfully. Result: {result}"
            except Exception as e:
                return f"Error executing tool '{tool_name}': {str(e)}"
        return f"Tool '{tool_name}' not found."

    @logfire.instrument()
    def run(self, user_input: str) -> str:
        """
        Verwerk gebruikersinvoer en genereer een reactie.
        """
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            system=self.system_prompt,
            messages=[{"role": "user", "content": user_input}],
        )

        # Extract the response text
        response_text = (
            response.content[0].text if hasattr(response, "content") else response
        )
        # Log the response to Logfire
        logfire.info(
            "Agent response generated",
            agent_name=self.name,
            user_input=user_input,
            response=response_text,
            model=self.model,
            response_length=len(response_text),
        )

        return response


basic_agent = LLMAgent(
    name="basic_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

Stel een vraag aan de agent

In [3]:
# Run the agent
result = basic_agent.run("geef een defintie van een agent")


print(result.content[0].text)

12:22:19.578 Calling __main__.LLMAgent.run
12:22:25.789   Agent response generated
Hier is een heldere definitie van een agent:

Een agent is een systeem of entiteit dat:

1. Waarnemingen kan doen in zijn omgeving
2. Doelgericht kan redeneren
3. Beslissingen kan nemen
4. Acties kan uitvoeren om specifieke doelen te bereiken

Er zijn verschillende soorten agenten:
- Softwareagenten (computerprogramma's)
- Fysieke agenten (robots)
- Menselijke agenten (bijvoorbeeld politieagenten)
- Intelligente agenten (systemen met AI)

Belangrijke kenmerken van een agent zijn:
- Autonomie
- Reactievermogen
- Pro-activiteit
- Sociale vaardigheid
- Adaptief vermogen

In de kunstmatige intelligentie wordt een agent gezien als een systeem dat waarnemingen ontvangt via sensoren en vervolgens handelingen verricht via actuatoren, met als doel het behalen van specifieke prestaties.


Definieer tools voor de agent

In [4]:
# Anthropic Tool Definitions (voor Pure Python)
ANTHROPIC_TOOLS = [
    {
        "name": "calculate_btw",
        "description": "Bereken BTW (belasting toegevoegde waarde) over een bedrag. Standaard BTW tarief is 21%.",
        "input_schema": {
            "type": "object",
            "properties": {
                "bedrag": {"type": "number", "description": "Het bedrag exclusief BTW"},
                "btw_percentage": {
                    "type": "number",
                    "description": "BTW percentage (standaard 21%)",
                    "default": 21.0,
                },
            },
            "required": ["bedrag"],
        },
    },
    {
        "name": "calculate_discount",
        "description": "Bereken korting over een bedrag.",
        "input_schema": {
            "type": "object",
            "properties": {
                "bedrag": {
                    "type": "number",
                    "description": "Het oorspronkelijke bedrag",
                },
                "korting_percentage": {
                    "type": "number",
                    "description": "Korting percentage",
                },
            },
            "required": ["bedrag", "korting_percentage"],
        },
    },
    {
        "name": "get_inwoners_gemeente",
        "description": "Haal inwonersaantallen op van Nederlandse gemeenten via CBS API. Kan specifieke gemeente opzoeken of top 10 grootste gemeenten tonen.",
        "input_schema": {
            "type": "object",
            "properties": {
                "gemeente_naam": {
                    "type": "string",
                    "description": "Naam van de gemeente (optioneel). Laat leeg voor top 10 grootste gemeenten.",
                },
                "jaar": {
                    "type": "string",
                    "description": "Jaar voor de data (standaard 2023)",
                    "default": "2023",
                },
            },
            "required": [],
        },
    },
]

# Tool function mapping
TOOL_FUNCTIONS = {
    "calculate_btw": tf.calculate_btw,
    "calculate_discount": tf.calculate_discount,
    "get_inwoners_gemeente": tf.get_inwoners_gemeente,
}

In [12]:
class ToolAgent(LLMAgent):
    def __init__(self, name: str, system_prompt: str = ""):
        super().__init__(name, system_prompt)
        self.tools = ANTHROPIC_TOOLS

    @logfire.instrument()
    def run(self, user_input: str) -> str:

        response = self.client.messages.create(
            model=self.model,
            max_tokens=1000,
            tools=self.tools,
            messages=[{"role": "user", "content": user_input}],
        )
        # Extract the response text
        response_text = (
            response.content[0].text if hasattr(response, "content") else response
        )
        # Log the response to Logfire
        logfire.info(
            "Agent response generated",
            agent_name=self.name,
            user_input=user_input,
            response=response_text,
            model=self.model,
            response_length=len(response_text),
        )

        # Check if Claude wants to use tools
        if response.stop_reason == "tool_use":
            tool_results = []

            for content in response.content:
                if content.type == "tool_use":
                    tool_name = content.name
                    tool_input = content.input
                    tool_id = content.id

                    print(
                        f"🔧 Claude gebruikt tool: {tool_name} met input: {tool_input}"
                    )

                    # Execute the tool
                    if tool_name in TOOL_FUNCTIONS:
                        result = TOOL_FUNCTIONS[tool_name](**tool_input)
                        tool_results.append(
                            {
                                "type": "tool_result",
                                "tool_use_id": tool_id,
                                "content": json.dumps(result),
                            }
                        )
                    else:
                        print(f"Tool {tool_name} not found")

            # Get final response with tool results
            final_response = self.client.messages.create(
                model=self.model,
                max_tokens=5000,
                tools=self.tools,
                messages=[
                    {"role": "user", "content": user_input},
                    {"role": "assistant", "content": response.content},
                    {"role": "user", "content": tool_results},
                ],
            )
            # Log the final response to Logfire
            logfire.info(
                "Agent response generated",
                agent_name=self.name,
                user_input=user_input,
                response=final_response,
            )
            return final_response
        else:
            return response


tool_agent = ToolAgent(
    name="tool_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

Test een aantal tool calls

In [13]:
result2 = tool_agent.run(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)

print(result2.content[0].text)

12:30:58.453 Calling __main__.ToolAgent.run
12:31:01.331   Agent response generated
🔧 Claude gebruikt tool: calculate_btw met input: {'bedrag': 100}
12:31:03.919   Agent response generated
Stap 2: Korting Berekening
Nu bereken ik de korting van 15% over het totale bedrag inclusief BTW.


In [14]:
result = tool_agent.run(
    "Geef een overzicht van de top-7 gemeenten in Nederland met het grootste aantal inwoners in 2025. Laat ook het aantal inwoners van Groningen zien."
)

print(result.content[0].text)

12:31:08.480 Calling __main__.ToolAgent.run
12:31:13.136   Agent response generated
🔧 Claude gebruikt tool: get_inwoners_gemeente met input: {'jaar': '2025'}
🔧 Claude gebruikt tool: get_inwoners_gemeente met input: {'gemeente_naam': 'Groningen', 'jaar': '2025'}
12:31:20.201   Agent response generated
Op basis van de gegevens voor het jaar 2025 kan ik je het volgende overzicht geven:

Top 7 gemeenten in Nederland naar aantal inwoners in 2025:
1. Amsterdam: 935.793 inwoners
2. Rotterdam: 672.330 inwoners
3. 's-Gravenhage (Den Haag): 568.419 inwoners
4. Utrecht: 376.435 inwoners
5. Eindhoven: 249.054 inwoners
6. Groningen: 244.807 inwoners
7. Tilburg: 230.359 inwoners

Specifiek voor Groningen: de gemeente Groningen zal in 2025 naar verwachting 244.807 inwoners hebben.

De prognose laat zien dat Amsterdam veruit de grootste gemeente blijft, met bijna 1 miljoen inwoners, terwijl Tilburg de top 7 completeert met ruim 230.000 inwoners.


In [15]:
result = tool_agent.run(
    "Als alle inwoners uit Eindhoven in 2025 15 euro betalen, hoeveel krijg ik dan.",
)
print(result.content[0].text)

12:31:38.301 Calling __main__.ToolAgent.run
12:31:41.909   Agent response generated
🔧 Claude gebruikt tool: get_inwoners_gemeente met input: {'gemeente_naam': 'Eindhoven', 'jaar': '2023'}
12:31:47.643   Agent response generated
Ik heb de gegevens gevonden. Eindhoven heeft momenteel 243.730 inwoners (peildatum 2023). 

Als elk van deze inwoners 15 euro zou betalen, dan zou de totale som zijn:
243.730 × 15 = 3.655.950 euro

Dus als alle inwoners van Eindhoven in 2025 15 euro betalen, zou je 3.655.950 euro krijgen.

Belangrijke opmerking: Dit is een hypothetische berekening. In de praktijk zou zoiets uiteraard niet zomaar kunnen plaatsvinden zonder nadere context of wettelijke grondslag.


Voeg geheugen toe aan de agent

In [10]:
class MemAgent(ToolAgent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.memory = []

    def add_memory(self, role, memory):
        self.memory.append({"role": role, "content": memory})

    def get_memory(self):
        return self.memory

    @logfire.instrument()
    def run(self, user_input):
        self.memory.append({"role": "user", "content": user_input})
        input = ""
        for message in self.memory:
            input += f"role': {message['role']}, 'content': {message['content']}\n"

        response = super().run(input)
        self.memory.append({"role": "assistant", "content": response.content[0].text})
        return response


mem_agent = MemAgent(
    name="mem_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

In [11]:
result = mem_agent.run(
    "ik koop een product X voor 100 euro exclusief BTW, bereken de BTW. Ik krijg van de leverancier ook nog 15% korting op het bedrag inclusief BTW. Geef een overzicht wat ik moet betalen en hoe dat is samengesteld"
)

print(result.content[0].text)

12:30:21.843 Calling __main__.MemAgent.run
12:30:21.844   Calling __main__.ToolAgent.run
12:30:26.044     Agent response generated
🔧 Claude gebruikt tool: calculate_btw met input: {'bedrag': 100}
12:30:28.488     Agent response generated
Stap 2: Bereken de korting op het bedrag inclusief BTW


In [ ]:
result = mem_agent.run(
    "wat was ook alweer het btw bedrag van het eerder gekochte product X en hoeveel korting kreeg ik. Zoek dit uit de conversatie hiervoor."
)

print(result.content[0].text)

Gebruik een MCP server om de tools te gebruiken.

In [ ]:
from mcp import ClientSession
from mcp.client.sse import sse_client
from contextlib import ExitStack
import requests
import asyncio


class MCPAgent(LLMAgent):

    def __init__(self, *args, **kwargs):
        """
        Initialiseer de Anthropic MCP client
        """
        super().__init__(*args, **kwargs)
        # Initialize session and client objects

    @logfire.instrument()
    async def run(self, user_input):

        # connect to the server via SSE
        async with sse_client("http://localhost:8050/sse") as (
            read_stream,
            write_stream,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                # initialize the session
                await session.initialize()
                print("Available tools:")
                tools_result = await session.list_tools()
                for tool in tools_result.tools:
                    print(f"  - {tool.name}: {tool.description}")

                # convert to the anthropic format
                anthropic_tools = []
                for tool in tools_result.tools:
                    anthropic_tool = {
                        "name": tool.name,
                        "description": tool.description,
                        "input_schema": tool.inputSchema,
                    }
                    anthropic_tools.append(anthropic_tool)

                # set the tools
                self.tools = anthropic_tools

                # run the agent
                response = self.client.messages.create(
                    model=self.model,
                    max_tokens=3000,
                    system=self.system_prompt,
                    tools=self.tools,
                    messages=[{"role": "user", "content": user_input}],
                )

                # Check if Claude wants to use tools
                if response.stop_reason == "tool_use":
                    tool_results = []

                    for content in response.content:
                        if content.type == "tool_use":
                            tool_name = content.name
                            tool_input = content.input
                            tool_id = content.id

                            print(
                                f"🔧 Claude gebruikt tool: {tool_name} met input: {tool_input}"
                            )

                            # Execute the tool
                            tool_result = await session.call_tool(
                                tool_name, {**tool_input}
                            )

                            tool_results.append(
                                {
                                    "type": "tool_result",
                                    "tool_use_id": tool_id,
                                    "content": tool_result.content[0].text,
                                }
                            )

                # Get final response with tool results
                final_response = self.client.messages.create(
                    model=self.model,
                    max_tokens=1000,
                    tools=self.tools,
                    messages=[
                        {"role": "user", "content": user_input},
                        {"role": "assistant", "content": response.content},
                        {"role": "user", "content": tool_results},
                    ],
                )
                # Log the final response to Logfire
                logfire.info(
                    "Agent response generated",
                    agent_name=self.name,
                    user_input=user_input,
                    response=final_response,
                )

        # Extract the response text
        if final_response:
            response = final_response
        else:
            response = response

        response_text = (
            response.content[0].text if hasattr(response, "content") else response
        )
        # Log the response to Logfire
        logfire.info(
            "Agent response generated",
            agent_name=self.name,
            user_input=user_input,
            response=response_text,
            model=self.model,
            response_length=len(response_text),
        )
        return response


mcp_agent = MCPAgent(
    name="mcp_agent",
    system_prompt="je bent een agent die op alle vragen een verstandig antwoord geeft",
)

In [ ]:
result = await mcp_agent.run("wat is de btw van 100 euro")

print(result.content[0].text)

In [ ]:
result = await mcp_agent.run("Hoeveel inwoners heeft Rotterdam?")

print(result.content[0].text)